# Project 5

In addition to answering the bolded questions on Coursera, also attach your notebook, both as
`.ipynb` and `.html`.

In the following exercise, we will perform bootstrap and cross validation to find statistical values for two datasets.

In this assignment, we will be using PennGrader, a Python package built by a former TA for autograding Python notebooks. PennGrader was developed to provide students with instant feedback on their answer. You can submit your answer and know whether it's right or wrong instantly. We then record your most recent answer in our backend database. You will have 100 attempts per test case, which should be more than sufficient.

<b>NOTE：Please remember to remove the </b>

```python
raise notImplementedError
```
<b>after your implementation, otherwise the cell will not compile.</b>


A few things to note before we start
- You do not have to round your answers.
- Please make sure your answer's variable name do match what is provided in grader cells.

## Getting Setup
Please run the below cells to get setup with the autograder. If you need to install packages, please do it below!

In [46]:
# %%capture
# !pip install penngrader --user

In [47]:
# !pip install scikit-learn --user
# !pip install statsmodels

Let's try PennGrader out! Fill in the cell below with your PennID and then run the following cell to initialize the grader.

<font color='red'>Warning:</font> Please make sure you only have one copy of the student notebook in your directory in Codio upon submission. The autograder looks for the variable `STUDENT_ID` across all notebooks, so if there is a duplicate notebook, it will fail.

In [48]:
#PLEASE ENSURE YOUR STUDENT_ID IS ENTERED AS AN INT (NOT A STRING). IF NOT, THE AUTOGRADER WON'T KNOW WHO 
#TO ASSIGN POINTS TO YOU IN OUR BACKEND

STUDENT_ID = 49731093                   # YOUR 8-DIGIT PENNID GOES HERE
STUDENT_NAME = "Newman Ilgenfritz"     # YOUR FULL NAME GOES HERE

In [49]:
import penngrader.grader

grader = penngrader.grader.PennGrader(homework_id = 'ESE542_Online_Spring_2021_HW5', student_id = STUDENT_ID)

## Part A

First, we will use the Boston dataset to try running our own implementation of bootstrap. 

In [50]:
#Data Wrangling
import pandas as pd
import numpy as np

#Logistic Regression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression

#Plotting
import matplotlib.pyplot as plt

#Cross Validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

#Statistics
from scipy import stats

RANDOM_STATE=42
%matplotlib inline

In [51]:
from sklearn.datasets import load_boston
boston_dataset = load_boston()
data = pd.DataFrame(data = boston_dataset.data, columns = boston_dataset.feature_names)
data['MEDV'] = pd.Series(boston_dataset.target)
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Based on this dataset, provide an estimate for $\hat\mu$, the population mean of 'medv'. Name this variable `mu_hat`.

In [52]:
# Enter your code below

mu_hat = data['MEDV'].mean()
mu_hat

22.532806324110677

In [53]:
grader.grade(test_case_id = 'test_mu_hat', answer = mu_hat)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


2. Provide an estimate of the standard error of $\hat\mu$. Name this variable `se_mu_hat`>. Interpret this result in your Jupyter notebook. 

*Hint*: We can compute the standard error of the sample mean by dividing the <u>sample</u> standard deviation by the square root of the number of observations. When computing the sample standard deviation, one must consider the Bessel’s correction factor from statistics (more information on that [here](https://en.wikipedia.org/wiki/Bessel\%27s_correction))

In [54]:
import math
sd = data['MEDV'].std()
se_mu_hat = sd / math.sqrt(len(data)-1) # -1 Bessel correction
se_mu_hat

0.4092657603105486

In [55]:
grader.grade(test_case_id = 'test_se_mu_hat', answer = se_mu_hat)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


3. Now estimate the standard error of  by implementing your own bootstrap method. Name your final answer `se_boostrap`. How does this compare to your answer from Part A Question 2? 

*Hint*: Generate $M=n$ artificial datasets, each with $N=n$ bootstrap samples. Use `np.random.seed(RANDOM_STATE)` before sampling so that your outputs stay consistent. You should try to write your own bootstrap method and and then compare it to packages such as SKLearn's `resample`.

In [56]:
from sklearn.utils import resample

np.random.seed(RANDOM_STATE)

n = data.shape[0]
print('n: ', n, '\n')

sampSize = n
samp = np.random.choice(data['MEDV'], size=sampSize, replace=True)

muSamp = samp.mean()
sdSamp = samp.std()

se = sdSamp/math.sqrt(len(samp)-1)

iterations = sampSize
means = []
for sets in range(iterations):
     
     samp = np.random.choice(data['MEDV'], size=sampSize, replace=True)
     means.append(np.mean(samp))


means.sort()

muSamp = np.asarray(means).mean()
print('muSamp: ', muSamp, '\n')

se_bootstrap = np.asarray(means).std(ddof=1)
print('se_bootstrap: ',se_bootstrap , '\n')

p = 0.95

u_pval = (1+p)/2.
l_pval = (1-u_pval)
l_indx = int(np.floor(n*l_pval))
u_indx = int(np.floor(n*u_pval))

ciTup = (means[l_indx], means[u_indx])
ciL = muSamp - (2*se_bootstrap)
ciU = muSamp + (2*se_bootstrap)


n:  506 

muSamp:  22.529964145667016 

se_bootstrap:  0.40631291586573504 



In [57]:
grader.grade(test_case_id = 'test_se_bootstrap', answer = se_bootstrap)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


4. Based on your bootstrap estimate from Part A Question 3, provide a $95\%$ confidence interval for the mean of 'medv'. Enter the lower range, higher range for mean's confidence interval in CI_lower, CI_upper.

*Hint*: You can approximate a $95\%$ confidence interval using the formula $[\hat\mu \pm 2 * SE(\hat\mu)] $.

In [58]:
CI_lower = ciL
CI_upper = ciU

In [59]:
grader.grade(test_case_id = 'test_CI_mean', answer = (CI_lower, CI_upper))

Correct! You earned 1.0/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


5. Based on this data set, provide the population median value of 'medv', $\widehat{median}$. Name this variable `median_hat`.

In [60]:
median_hat = data['MEDV'].median()
median_hat

21.2

In [61]:
grader.grade(test_case_id = 'test_median_hat', answer = median_hat)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


6. We now would like to estimate the standard error of $\widehat{median}$. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using bootstrap. Name this variable `se_bootstrap_median`.

*Hint*: use your bootstrap sample list from A.3.

In [62]:
medians = []
for sets in range(iterations):
    
     samp = np.random.choice(data['MEDV'], size=sampSize, replace=True)
     medians.append(np.median(samp))

medians.sort()

muSamp = np.asarray(medians).mean()

se_bootstrap_median = np.asarray(means).std(ddof=1)

p = 0.95

u_pval = (1+p)/2.
l_pval = (1-u_pval)
l_indx = int(np.floor(n*l_pval))
u_indx = int(np.floor(n*u_pval))

ciTup = (means[l_indx], means[u_indx])
ciL = muSamp - (2*se_bootstrap)
ciU = muSamp + (2*se_bootstrap)

In [63]:
grader.grade(test_case_id = 'test_se_median', answer = se_bootstrap_median)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


7.  Is the standard error of the median greater or less than the standard error of the mean?Does this make intuitive sense? Comment on your other findings in the cell below

In [65]:
delta = se_bootstrap - se_bootstrap_median
delta
if(delta>0):
    print('standard error of mean is greater than standard error of median')
else:
    print('standard error of median is greater than standard error of mean')

standard error of median is greater than standard error of mean


## Part B

Next, we will use the Default dataset to predict the probability of default using income and balance. In doing so, we also want to estimate the test error of the logistic regression model described in that section using cross validation.

In [66]:
default = pd.read_csv('Default.csv')
encode = lambda x: 1 if x == 'Yes' else 0
default['default'] = default['default'].map(encode)
default['student'] = default['student'].map(encode)
default.head(5)

,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879


1. Fit a logistic regression model that use <u>all</u> the data from the predictors 'income' and 'balance' to predict 'default'. Please name your model <b>fit1</b>.

*Hint*: Use `Stats Models` to fit the logistic regression. Review the previous recitations for review.

In [67]:
fit1 = smf.glm('default~balance+income', data = default, family=sm.families.Binomial()).fit()
print(fit1.summary(), '\n')
print('Predicted probability of default:',fit1.predict(), '\n') #Predict

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Wed, 24 Feb 2021   Deviance:                       1579.0
Time:                        16:48:53   Pearson chi2:                 6.95e+03
No. Iterations:                     9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.5405      0.435    -26.544      0.0

In [68]:
grader.grade(test_case_id ='test_glm_type', answer = str(type(fit1)))

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [69]:
grader.grade(test_case_id ='test_glm', answer = fit1.params)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


2. Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps: 


- Split the sample set into a random training set and a random validation set. Use a test size of 20%. *Hint*: Use `sklearn.model_selection.train_test_split` and a `random_state` of 42/RANDOM_STATE. Store your training set in <b>train</b>, testing set in <b>test</b>.

In [70]:

train, test = train_test_split(default, test_size=0.20, random_state=RANDOM_STATE)
print('train head: ')
print(train.head(), '\n')
print('test head: ')
print(test.head(), '\n')

train head: 
      default  student      balance        income
9254        0        0  1018.568130  34103.879520
1561        0        0    62.170050  28660.747508
1670        0        0  1046.743543  40822.447413
6087        0        0   763.735280  44125.718725
6669        0        1   697.248633  25730.917583 

test head: 
      default  student      balance        income
6252        0        0  1435.662933  31507.089277
4684        0        0   771.789347  42139.070269
1731        0        0     0.000000  21809.218509
4742        0        0   113.571264  32803.832648
4521        0        0  1358.132472  49903.597081 



In [71]:
grader.grade(test_case_id ='test_train_shape', answer = train)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [72]:
grader.grade(test_case_id ='test_test_shape', answer = test)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- Fit a logistic regression with multiple variables model using <u>only</u> the training observations. Name your model <b>fit2</b>.

In [73]:
logit_equation = 'default~balance+income'
fit2 = smf.glm(logit_equation, data = train, family=sm.families.Binomial()).fit()
print(fit2.summary(), '\n')

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:                 8000
Model:                            GLM   Df Residuals:                     7997
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -609.84
Date:                Wed, 24 Feb 2021   Deviance:                       1219.7
Time:                        16:49:03   Pearson chi2:                 6.64e+03
No. Iterations:                     9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.8297      0.506    -23.387      0.0

In [74]:
grader.grade(test_case_id ='test_glm_type', answer = str(type(fit2)))

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [75]:
grader.grade(test_case_id ='test_train_para', answer = fit2.params)

Correct! You earned 1.0/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- Obtain a prediction of 'default' for each individual in the test set by computing the probability of default for that individual and classifying the individual to the default category if the posterior probability is greater than or equal to 0.5 (a Bayesian classifier). Store your predictions in <b>predicted</b>.

*Hint*: we check for both the size and values of your precitions!

In [76]:
# Enter your code below
#print('Predicted probability of default:',fit2.predict(), '\n') #Predict
Xtest = pd.DataFrame()
Xtest['balance'] = test['balance']
Xtest['income'] = test['income']
print('Xtest.head: ')
print(Xtest.head(), '\n')
print('Xtest size: ', len(Xtest), '\n')

#encode = lambda x: 1 if x == 'Yes' else 0
ytest = test['default']
yprobs = fit2.predict(Xtest)
yPred =  [0 if y < 0.5 else 1 for y in yprobs]

predicted = np.asarray(yPred)

from sklearn.metrics import confusion_matrix
x = confusion_matrix(ytest, yPred)


from sklearn.metrics import accuracy_score
aScore = accuracy_score(ytest, yPred)


fraction_correct_test = aScore

# Your answer should be the ratio of number of correct predictions over the total number of samples.
#mis_rate = 1.0 - aScore

#mis_rate = float(28/2000)
mis_rate = x[0,1]/len(predicted)

print('mis_rate: ')
print(mis_rate, '\n')



#raise NotImplementedError

Xtest.head: 
          balance        income
6252  1435.662933  31507.089277
4684   771.789347  42139.070269
1731     0.000000  21809.218509
4742   113.571264  32803.832648
4521  1358.132472  49903.597081 

Xtest size:  2000 

mis_rate: 
0.005 



In [77]:
grader.grade(test_case_id ='test_glm_predict', answer = predicted)

Correct! You earned 1.0/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- Compute the misclassification rate, which is the fraction of the observations in the validation set that are misclassified. Name this variable `mis_rate` and. 

*Hint*: Compare the predictions with the test set. Your answer should be the ratio of number of correct predictions over the total number of samples.

In [78]:
mis_rate = 1-aScore

In [79]:
grader.grade(test_case_id ='test_mis_rate', answer = mis_rate)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


3. Repeat the process in Part B Question 2 three times, using three different splits of the observations into a training set and a validation set. Use test sizes of $10\%$, $30\%$, and $50\%$. Name the misclassification rates `mis_rate_10`, `mis_rate_30`, `mis_rate50`, respectively. Comment on the results obtained and determine the key takeaway. Store the accuracy score for each model within <b>model_rates</b> and model parameters within <b>model_params</b>

*Note*: When splitting to test/train sets, please set your random_state to RANDOM_STATE so that you can have the same group of testing and training sets.

In [80]:
testSize10 = 0.10
testSize30 = 0.30
testSize50 = 0.50
testList = [testSize10, testSize30, testSize50]
fitList = []
predict = []
aScore = []
mis_rate = []
model_rates = []

for i in range(3):
    train, test= train_test_split(default, test_size=testList[i], random_state=RANDOM_STATE)
    fitList.append(smf.glm(logit_equation, data = train, family=sm.families.Binomial()).fit())
    Xtest = pd.DataFrame()
    Xtest['balance'] = test['balance']
    Xtest['income'] = test['income']
 
    ytest = test['default']
    
    yprobs = fitList[i].predict(Xtest)
   
    yPred =  [0 if y < 0.5 else 1 for y in yprobs]

    predict.append(np.asarray(yPred))
  
    x = confusion_matrix(ytest, yPred)
    
    aScore.append(accuracy_score(ytest, yPred))

    mis_rate.append(1.0 - aScore[i])

model_rates = mis_rate
print('model_rates: ')
print(model_rates, '\n')


model_rates: 
[0.03300000000000003, 0.026666666666666616, 0.025800000000000045] 



In [81]:
grader.grade(test_case_id ='test_models_rates', answer = model_rates)

Correct! You earned 2.0/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


4. Using KFold cross validation with five folds across 5 trials, calculate the <b>average</b> misclassification rate. Name this `mis_rate_kfold`. In your notebook, briefly explain the KFold process. 

*Hint*: Because `Stats Models` does not have its own cross validation libraries, you may have to use the following packages (specify a `random_state` equaling the trial number(5)): 


- `sklearn.linear_model.LogisticRegression`
- `sklearn.model_selection.cross_val_score`
- `sklearn.model_selection.KFold`

In [82]:
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

blue = '#008FD5'
red = '#FF2700'
green = '#77AB43'
plt.style.use('ggplot')

def poly(x, p):
    x = np.array(x)
    X = np.transpose(np.vstack([x**k for k in range(p+1)]))
    return np.linalg.qr(X)[0][:,1:]

X = pd.DataFrame()
X['balance'] = default['balance']
X['income']  = default['income']

y = default['default']

model = LinearRegression() #Define ML Model

score = np.array([])# array to store errors
order = np.array([]) # array to store order of polynomial
trials = 5
splits = 10


for trial in range(trials):
    cv_method = KFold(n_splits=splits,shuffle=True,random_state = trials) #Define CV method
    error = -1*np.mean(cross_val_score(model,X,y,cv = cv_method,scoring = 'neg_mean_squared_error'))
    score = np.append(score, error)

mis_rate_kfold = score.mean()
print('mis_rate_kfold: ',mis_rate_kfold ,'\n')

mis_rate_kfold:  0.028244708096504846 



In [83]:
grader.grade(test_case_id ='test_mis_kfold', answer = mis_rate_kfold)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


5. Using bootstrap on the complete dataset, compute estimates for the standard errors of the `income` and `balance` logistic regression coefficients. In order to do this, you should perform the following steps:


- Write a function that takes as input the Default data set (and optionally, an index of the observations). The method should be called `boot_sample(data,index)`. The method should return the coefficient estimates for income and balance in the logistic regression model.

In [84]:
def boot_sample(data, index):
    
    return resample(data)

b. Use this function to estimate the standard errors of the logistic regression coefficients for income and balance using 100 different bootstrap samples. Store your lists of results in boot_income, boot_balance, both of lists should be of size (100,1). 
*Note*: Do not set a random seed in this problem, otherwise you will not be randomly sample from the data.

In [85]:
length = 100
boot_income = np.array([])
boot_balance = np.array([])

from statsmodels.discrete.discrete_model import Logit

intercept = []
income = []
balance = []

index = 0
for trial in range(length):
    train = boot_sample(default, index)
    X = train[['income', 'balance']]
    X = sm.add_constant(X, prepend=True)
    y = train['default']

    model = Logit(y, X)
    result = model.fit(disp=False)
    intercept.append(result.params.const)
    income.append(result.params.income)
    balance.append(result.params.balance)

boot_balance = balance
boot_income = income

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [86]:
grader.grade(test_case_id ='test_boot_income_balance', answer = (boot_income, boot_balance))

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- Comment on the estimated standard errors obtained. What are the confidence intervals of the logistic regression coefficients? 

In [87]:
"""
    Estimated SEs indicate high confidence there is a significant relationship
    between default rates and both balance and income 
    
    confidence intervals:
           balance:     0.005      0.006
           income:      1.1e-05    3.06e-05

"""

'\n    Estimated SEs indicate high confidence there is a significant relationship\n    between default rates and both balance and income \n    \n    confidence intervals:\n           balance:     0.005      0.006\n           income:      1.1e-05    3.06e-05\n\n'

Congratulations on making the end of project 5. Please remember to save a copy of .ipynb and .html of your notebook, and mark your project as complete on Codio.